In [7]:
# Import dependencies
import pandas as pd
import numpy as np
import time
import re

In [44]:
# Create DataFrames from CSV file
t0 = time.time()
nlp_df = pd.read_csv('../Data/nlp_df.csv')
lyric_TF_df = pd.read_csv('../Data/lyric_TF.csv')
cooccurring_words_df = pd.read_csv('../Data/top_cooccurring_words.csv')
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
display(nlp_df.head(3))
display(lyric_TF_df.head(3))
cooccurring_words_df.head(3)

Run time: 130.79740118980408 seconds


,song,song_id,artist,artist_id,category,category_id,genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,...,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,non_alpha_words,words,filtered,language
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,8,"['dance', 'pop']",0.392,0.574,7.0,...,0.145,0.529,81.112,214707.0,4.0,Im like the water when your ship rolled in th...,1,"['', 'im', 'like', 'the', 'water', 'when', 'yo...","['', 'im', 'like', 'water', 'ship', 'rolled', ...","(1, 'en')"
1,Stay Next To Me (with Chelsea Cutler),6SGG5AxHShqSYiV9fCWpZz,Quinn XCII,3ApUX1o6oSz321MMECyIYd,pop,8,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,...,0.366,0.756,179.954,206046.0,4.0,Didnt even wanna go out whyd you call me ? Iv...,1,"['', 'didnt', 'even', 'wanna', 'go', 'out', 'w...","['', 'didnt', 'even', 'wanna', 'go', 'whyd', '...","(1, 'en')"
2,WITHOUT YOU,27OeeYzk6klgBh83TSvGMA,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,pop,8,['australian'],0.662,0.413,0.0,...,0.134,0.467,93.005,161385.0,4.0,You cut out a piece of me and now I bleed int...,1,"['', 'you', 'cut', 'out', 'a', 'piece', 'of', ...","['', 'cut', 'piece', 'bleed', 'internally', 'l...","(1, 'en')"


,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,...,corners,petroleum,amoebaaaaaaaa,misanthropical,hoof,taunted,interlude,sting,axis,employed
0,NaN,NaN,NaN,53368.0,NaN,4518.1163,5348.9565,42484.0,-60353.306,5500.0,...,31.0,4.0,8.0,2.0,2.0,2.0,7.0,49.0,4.0,3.0
1,willow,Taylor Swift,pop,8.0,"['dance', 'pop']",0.3920,0.5740,7.0,-9.195,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,"['indie', 'pop', 'electropop']",0.5810,0.5840,2.0,-4.928,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,oh oh,yeah yeah,ooh ooh,na na,dont know,la la,dont wanna,let go,im gonna,oh yeah,...,good time,dont mind,last night,cause youre,dont worry,brand new,like dont,know like,im feeling,feel good
0,4770,3231,2572,2425,2384,2343,1519,1238,1208,1075,...,260,258,257,257,256,255,254,253,251,250


In [45]:
# Create a list of all words, unique word counts, and filtered words
t0 = time.time()
words_list = []
unique_word_counts = []
filtered_words_list = []
for index, row in nlp_df.iterrows():
    filtered_words = row['filtered']
    filtered_words = filtered_words.replace(',', '').replace("'", '')
    filtered_words = filtered_words.replace('[', '').replace(']', '')
    filtered_words = filtered_words.replace('#', '').replace('&nbsp', '')
    filtered_words = filtered_words.replace('?', '? ').replace(',', '')
    filtered_words = filtered_words.replace('/', ' ')
    filtered_words = filtered_words.replace('\\u200a', '').replace('\\u200b', '')
    filtered_words = filtered_words.replace('\\u2063', '').replace('\u202f', '')
    filtered_words = filtered_words.replace('\\u2028', ' ').replace('\\u2008', ' ')
    while ('\\u200e' in filtered_words) or ('\\xa0' in filtered_words):
        filtered_words = filtered_words.replace('\\u200e', '')
        filtered_words = filtered_words.replace('\\xa0', '')
    filtered_words_list.append(filtered_words)
    unique_words = list(set(filtered_words.strip().split(' ')))
    unique_word_counts.append(len(unique_words))
    words_list.extend(unique_words)
word_columns = list(set(words_list))
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
len(word_columns)

Run time: 1.5111579895019531 seconds


39244

In [46]:
# Add a unique_word_count column and replace the filtered column
nlp_df['unique_word_count'] = unique_word_counts
nlp_df['filtered'] = filtered_words_list
nlp_df.head(1)

,song,song_id,artist,artist_id,category,category_id,genres,audio_ft_danceability,audio_ft_energy,audio_ft_key,...,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,non_alpha_words,words,filtered,language,unique_word_count
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,8,"['dance', 'pop']",0.392,0.574,7.0,...,0.529,81.112,214707.0,4.0,Im like the water when your ship rolled in th...,1,"['', 'im', 'like', 'the', 'water', 'when', 'yo...",im like water ship rolled night rough surface...,"(1, 'en')",87


In [47]:
# Remove songs with less than 25 unique words
nlp_df = nlp_df[nlp_df['unique_word_count']>=25]
len(nlp_df)

8115

# Try to remove non English songs

In [48]:
# Remove non English songs
songs_to_keep = [78, 452, 715, 878, 1633, 2150, 2402, 2407, 2427, 2526, 2835, 2979, 3232,
                 3359, 3791, 4368, 4643, 5692, 6328, 6379, 6387, 6523, 6722, 7853]
songs_to_remove = [264, 924, 1610, 1618, 2299, 2368, 3688, 5482, 5656, 5658, 5688, 5695, 
                   5710, 5715, 5719, 5720, 5750, 5792, 6385]
songs_to_remove.extend([item for item in one_lang_not_en if item not in songs_to_keep])
nlp_df = nlp_df.drop(songs_to_remove)
print(len(songs_to_remove))
len(nlp_df)

100


8015

In [61]:
# Add co-occurring word columns
t0 = time.time()
cooccurring_words = list(cooccurring_words_df.columns)
for words in cooccurring_words:
    lyric_TF_df[words] = 0
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
len(lyric_TF_df.columns)

Run time: 0.04604697227478027 seconds


21646

In [62]:
# Add the term frequencies to the DataFrame
t0 = time.time()
nlp_df2 = nlp_df.reset_index()
uncommon_cooccurring_words = []
for index, row in lyric_TF_df.iterrows():
    lyrics = nlp_df2['filtered'][index].strip().split(' ')
    while '' in lyrics:
        lyrics.remove('')
    while ' ' in lyrics:
        lyrics.remove(' ')
    for i in range(len(lyrics)-1):
        co_occurring_words = lyrics[i] + ' ' + lyrics[i+1]
        if co_occurring_words in cooccurring_words:
            lyric_TF_df.loc[index, co_occurring_words]+=1
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
lyric_TF_df.head(3)

Run time: 118.53468108177185 seconds


,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,...,good time,dont mind,last night,cause youre,dont worry,brand new,like dont,know like,im feeling,feel good
0,NaN,NaN,NaN,53368.0,NaN,4518.1163,5348.9565,42484.0,-60353.306,5500.0,...,0,0,0,0,0,0,0,0,0,0
1,willow,Taylor Swift,pop,8.0,"['dance', 'pop']",0.3920,0.5740,7.0,-9.195,1.0,...,0,0,0,0,0,0,0,0,0,0
2,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,"['indie', 'pop', 'electropop']",0.5810,0.5840,2.0,-4.928,1.0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
# Create new total row
lyric_TF_df = lyric_TF_df.drop(0)
lyric_TF_df.loc[-1] = lyric_TF_df.sum(numeric_only=True)
lyric_TF_df.index = lyric_TF_df.index + 1  
lyric_TF_df = lyric_TF_df.sort_index()
lyric_TF_df.head(3)

,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,...,good time,dont mind,last night,cause youre,dont worry,brand new,like dont,know like,im feeling,feel good
0,NaN,NaN,NaN,53368.0,NaN,4518.1163,5348.9565,42484.0,-60353.306,5500.0,...,259.0,252.0,257.0,251.0,256.0,255.0,254.0,253.0,239.0,223.0
2,willow,Taylor Swift,pop,8.0,"['dance', 'pop']",0.3920,0.5740,7.0,-9.195,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,"['indie', 'pop', 'electropop']",0.5810,0.5840,2.0,-4.928,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:
# Create filtered DataFrame without columns of words appear less than 4 times
filtered_lyric_TF_df = lyric_TF_df.drop(columns=lyric_TF_df.columns[(lyric_TF_df[0:1]<4).any()])    
filtered_lyric_TF_df.head(3)

,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,audio_ft_speechiness,...,good time,dont mind,last night,cause youre,dont worry,brand new,like dont,know like,im feeling,feel good
0,NaN,NaN,NaN,53368.0,NaN,4518.1163,5348.9565,42484.0,5500.0,664.1198,...,259.0,252.0,257.0,251.0,256.0,255.0,254.0,253.0,239.0,223.0
2,willow,Taylor Swift,pop,8.0,"['dance', 'pop']",0.3920,0.5740,7.0,1.0,0.1700,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,"['indie', 'pop', 'electropop']",0.5810,0.5840,2.0,1.0,0.2840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
# Save DataFrames to CSV
lyric_TF_df.to_csv('../Data/cooccurring_lyric_TF.csv', index=False)
filtered_lyric_TF_df.to_csv('../Data/filtered_coocurring_TF.csv', index=False)

In [ ]:
# Make the song name be the index of the DataFrame